# Rapid Exploration

In [ ]:
%matplotlib inline
#%matplotlib notebook

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython.display import Image
from sklearn.cluster import KMeans
import  PIL
import cv2 as cv
from scipy import ndimage, misc
import cv2

In [ ]:
IMAGE_PATH = '../data/images/green_screen_ordered.jpg'

In [ ]:
plt.rcParams['figure.figsize'] = [15, 15]

## Load example image and display it

In [ ]:
Image(IMAGE_PATH, width=500)

# Convert to hsv

In [ ]:
img = cv.imread(IMAGE_PATH)
hsv = cv.cvtColor(img, cv.COLOR_BGR2HSV)

In [ ]:
hist = cv.calcHist([hsv], [0,1], None, [180, 256], [0,180,0,256])
plt.plot(hist, color='k')
plt.show()

In [ ]:
for i, col in enumerate(['b', 'g', 'r']):
    hist = cv.calcHist([img], [i], None, [256], [0, 256])
    plt.plot(hist, color = col)
    plt.xlim([0, 256])
    
plt.show()

## Histogram to try to isolate green screen

In [ ]:
def show_img_compar(img_1, img_2 ):
    f, ax = plt.subplots(1, 2, figsize=(10,10))
    ax[0].imshow(img_1)
    ax[1].imshow(img_2)
    ax[0].axis('off') #hide the axis
    ax[1].axis('off')
    f.tight_layout()
    plt.show()

In [ ]:
def palette(clusters):
    width=300
    palette = np.zeros((50, width, 3), np.uint8)
    steps = width/clusters.cluster_centers_.shape[0]
    for idx, centers in enumerate(clusters.cluster_centers_): 
        palette[:, int(idx*steps):(int((idx+1)*steps)), :] = centers
    return palette

clt_3 = KMeans(n_clusters=3)
clt_3.fit(img.reshape(-1, 3))
show_img_compar(img, palette(clt_3))

# Green screening

In [ ]:
src = cv.imread(IMAGE_PATH)
imgRGB = cv.cvtColor(src, cv.COLOR_BGR2RGB)
imgHSV = cv.cvtColor(imgRGB, cv.COLOR_BGR2HSV)

imgRGB = ndimage.rotate(imgRGB, 90)
imgHSV = ndimage.rotate(imgHSV, 90)

In [ ]:
hist = cv.calcHist([imgHSV], [0,1], None, [180, 256], [0,180,0,256])
plt.plot(hist, color='k')
plt.show()

In [ ]:
for i, val in enumerate(np.max(hist[30:60,:], axis=1)):
    print(f"{30+i}: {val}")

In [ ]:
plt.rcParams['figure.figsize'] = [15, 12]

lower = np.array([35, 50, 0])    #Lower values of HSV range; Green have Hue value equal 120, but in opencv Hue range is smaler [0-180]
upper = np.array([52, 255, 255])  #Uppervalues of HSV range
imgRange = cv.inRange(imgHSV, lower, upper)
flipMaskImg = cv.bitwise_not(imgRange)

# Adjust the range threshold.
alteredMask = np.array(flipMaskImg, copy=True)
alteredMask = cv.dilate(alteredMask, np.ones((3,3),np.uint8), 1)
alteredMask = cv.erode(alteredMask, np.ones((3,3),np.uint8), 1)

alteredMask = cv.bitwise_not(alteredMask)

# Construct masked images.
rawMaskApplied = cv.bitwise_and(imgRGB, imgRGB, mask = imgRange)
alteredMaskApplied = cv.bitwise_and(imgRGB, imgRGB, mask = alteredMask)

# Generate rough contours.
contourImg = np.array(alteredMaskApplied, copy=True)
contours, hierarchy = cv.findContours(alteredMask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

# Filter out contours too big or small.
med = np.median([cv.contourArea(c) for c in contours])
contours_trimmed = []
for c in contours:
    con_area = cv.contourArea(c)
    if con_area < 2*med and con_area > 0.5*med:
        contours_trimmed.append(c)

cv.drawContours(contourImg, contours_trimmed, -1, (0,255,0), 3)

print(len(contours_trimmed))


plt.figure(1)

ax1 = plt.subplot(221)
plt.imshow(alteredMaskApplied)
ax1.title.set_text('alteredMaskApplied')


ax2 = plt.subplot(222)
plt.imshow(contourImg)
ax2.title.set_text('contourImg')

ax3 = plt.subplot(223)
plt.imshow(rawMaskApplied)
ax3.title.set_text('rawMaskApplied')

ax4 = plt.subplot(224)
plt.imshow(imgRGB)
ax4.title.set_text('imgRGB')

plt.show()



# Finding points

In [ ]:
labeledPieces = np.copy(contourImg)


for (i,c) in enumerate(contours_trimmed):
    M= cv2.moments(c)
    cx= int(M['m10']/M['m00'])
    cy= int(M['m01']/M['m00'])
    cv2.putText(labeledPieces, text= str(i+1), org=(cx-20,cy+10),
            fontFace= cv2.FONT_HERSHEY_SIMPLEX, fontScale=0.8, color=(255,0,255),
            thickness=2, lineType=cv2.LINE_AA)

plt.imshow(labeledPieces)
plt.show()

# Analyzing contour output

In [ ]:
# How deep it goes
contours_trimmed[0][0][0][1]

In [ ]:
# What the dimensions mean
# contours_trimmed[0][0][0][1]
# 63 length pieces, 244 points for piece 0, [x, y dim]
contours_trimmed[0][0]

In [ ]:
# Top and bottom 5 number of points in contours
s = sorted([len(c) for c in contours_trimmed])
s[0:5] + s[-6: -1]

# Find bounding boxes of contours

In [ ]:
rects = [cv.boundingRect(cnt) for cnt in contours_trimmed]
# pushes tuples of (x,y,w,h)

# Let's expand rects.
rects = [(x-10, y-10, w + 20, h + 20) for (x,y,w,h) in rects]

rects[0:3]

In [ ]:
imageLabeledWRects = np.copy(labeledPieces)
for (x,y,w,h) in rects:
    cv.rectangle(imageLabeledWRects,(x,y),(x+w,y+h),(255,100,0),2)
plt.imshow(imageLabeledWRects)
plt.show()

# Contour to polygon explorer

* Corner: 3
* Edge: 26, 58
* Center pieces: 14, 31, 38, 45, 48

TODO:
Create 4 image plot that takes contour index and displays:
* untouched puzzle piece
* 3 other images with unique epsilons

### Let's try to extract a puzzle pieces and show it

In [ ]:
x, y, w, h = rects[0]

piece = labeledPieces[y:y+h, x:x+w, :]

plt.imshow(piece)
plt.show()

In [ ]:
index = 0
pieces = [3,   26, 58,    14, 31, 38, 45, 48]


def renderPiecePolys(index):
    pieceIndex = pieces[index]

    pieceContour, (x,y,w,h) = contours_trimmed[pieceIndex], rects[pieceIndex]
    lenContour = cv2.arcLength(pieceContour, True)
    piece = labeledPieces[y:y+h, x:x+w, :]


    plt.figure(1)

    ax1 = plt.subplot(231)
    plt.imshow(piece)
    ax1.title.set_text(f'Default image - Points: {len(pieceContour)}')

    levels = [.005, .01, .02, .03, .05]
    count = 0
    for i, ep in enumerate(levels):
        sp = 232 + i
        ax = plt.subplot(sp)
        approxContour = cv2.approxPolyDP(pieceContour, ep * lenContour, True)
        subPiece = np.copy(piece)
        subPiece = cv2.drawContours(subPiece, [approxContour], 0, (255,0,255), offset = (-x,-y))
        ax.imshow(subPiece)
        ax.title.set_text(f'Epsilon percent: {ep} - Points: {len(approxContour)}')
        count += 1

    plt.show()


In [ ]:
renderPiecePolys(2)